This Notebook will be dedicated to parsing the Play-By-Play data from RetroSheets. This will allow us to gather data on a per-player basis to give us many more features with the hope of deriving more meaningful results from the models we train. The goal for this parser will to give us a boxscore-like DataFrame for each game from 2014-2019. There will be 18 batters for both NL and AL games. For the sake of simplicity, pitching stats are not included in the PBP for now, but this can easily be changed for the future. Let's walk through how we'll parse this data.

We will go through each play of each game, and update statistics for each player after each play. 

The fields of each player will be:

- (Home/Visitor) Player i:
    - player id
    - hits
    - singles
    - doubles
    - triples
    - walks
    - bunts
    - sacrifice hits
    - sacrifice flys
    - RBIs
    - at bats
    - number of stolen bases
    - number times caught stealing
    - number times picked off
    - number of errors
    
Therefore each player has 15 fields associated with them, and since there will be 18 batters, there will be a total of 270 columns of batter data. An example of one of these columns is "Visiting Player i Errors" where "i" is the ith player on the visiting team, which is determined by the "batting" flag in the event file and "Errors" is determined by summing the number of errors that player committed over the course of a single game.

In addition to the 270 batter data columns, there will be one column for the game ID, which be used to get the stats on a per-game basis.

In [7]:
import pandas as pd

column_labels = ['Game ID', 
                 'Visiting Team', 
                 'Batting Team',
                 'Batter', 
                 'First Runner', 
                 'Second Runner',
                'Third Runner',
                'Event Type',
                'Batter Event Flag',
                'AB Flag',
                'Hit Value',
                'SH Flag',
                'SF Flag',
                'RBI On Play',
                'Wild Pitch Flag',
                'Passed Ball Flag',
                'Bunt Flag',
                'Foul Flag',
                '1st Error Player',
                '1st Error Type',
                '2nd Error Player',
                '2nd Error type',
                '3rd Error Player',
                '3rd Error Type',
                'SB For Runner On 1st Flag',
                'SB For Runner On 2nd Flag',
                'SB For Runner On 3rd Flag',
                'CS For Runner On 1st Flag',
                'CS For Runner On 2nd Flag',
                'CS For Runner On 3rd Flag',
                'PO For Runner On 1st Flag',
                'PO For Runner On 2nd Flag',
                'PO For Runner On 3rd Flag',
                'Event Num']

mets_2014 = pd.read_csv('datasets/retro_sheet_pbp_new/2014NYN.EVN', names=column_labels)

mets_2014.head()

,Game ID,Visiting Team,Batting Team,Batter,First Runner,Second Runner,Third Runner,Event Type,Batter Event Flag,AB Flag,...,SB For Runner On 1st Flag,SB For Runner On 2nd Flag,SB For Runner On 3rd Flag,CS For Runner On 1st Flag,CS For Runner On 2nd Flag,CS For Runner On 3rd Flag,PO For Runner On 1st Flag,PO For Runner On 2nd Flag,PO For Runner On 3rd Flag,Event Num
0,NYN201403310,WAS,0,spand001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,1
1,NYN201403310,WAS,0,zimmr001,NaN,NaN,NaN,20,T,T,...,F,F,F,F,F,F,F,F,F,2
2,NYN201403310,WAS,0,wertj001,zimmr001,NaN,NaN,3,T,T,...,F,F,F,F,F,F,F,F,F,3
3,NYN201403310,WAS,0,ramow001,zimmr001,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,4
4,NYN201403310,WAS,1,youne003,NaN,NaN,NaN,3,T,T,...,F,F,F,F,F,F,F,F,F,5


In [8]:
mets_2014.shape

(6263, 34)

In [48]:
# Let's start by getting Starting Lineups
import os

gl_path = 'datasets/retro_sheet_gls/'
sl_path = 'datasets/starting_lineups/'
ignore = ['.ipynb_checkpoints']

if 'starting_lineups' not in os.listdir('datasets/'):
    os.mkdir('starting_lineups')

for file in os.listdir(gl_path):
    if file not in ignore:
        gl = pd.read_csv(gl_path + file)
        gl['Game ID'] = gl['Home Team'].str.cat(
                gl['Date'].astype('string').str.cat(
                gl['Number of game'].astype('string')))
        starting_lineups = gl.filter(items=['Game ID'] + 
                                     [f'Visiting Team Player {i+1} ID' for i in range(9)] + 
                                     [f'Home Team Player {i+1} ID' for i in range(9)],
                                    axis=1)
        starting_lineups.to_csv(sl_path + 'SL' + file[2:6] + '.csv')

In [202]:
def calculate_stats(data, player_id, player_num, team):
    stats = dict()
    # calculate batting stats
    h, s, d, t, hr, w = 0, 0, 0, 0, 0, 0
    for event in data[data['Batter'] == player_id]['Event Type']:
        if event == 20:
            s, h = s+1, h+1
        elif event == 21:
            d, h = d+1, h+1
        elif event == 22:
            t, h = t+1, h+1
        elif event == 23:
            hr, h = hr+1, h+1
        elif event == 14 or event == 15:
            w += 1
    # calculate at-bats, sh, sh, RBIs and bunts
    tf = ['AB Flag', 'SH Flag', 'SF Flag', 'Bunt Flag']
    tf_val = [0,0,0,0]
    for label_index in range(len(tf)):
        tf_val[label_index] += data[(data['Batter'] == player_id) & (data[tf[label_index]] == 'T')].shape[0]
    ab, sh, sf, b = tf_val
    rbi = sum(data[data['Batter'] == player_id]['RBI On Play'])
    # calculate base running stats
    sb, cs, po = 0, 0, 0
    sb += data[(player_id == data['First Runner']) & ('T' == data['SB For Runner On 1st Flag'])].shape[0]
    cs += data[(player_id == data['First Runner']) & ('T' == data['CS For Runner On 1st Flag'])].shape[0]
    po += data[(player_id == data['First Runner']) & ('T' == data['PO For Runner On 1st Flag'])].shape[0]

    sb += data[(player_id == data['Second Runner']) & ('T' == data['SB For Runner On 2nd Flag'])].shape[0]
    cs += data[(player_id == data['Second Runner']) & ('T' == data['CS For Runner On 2nd Flag'])].shape[0]
    po += data[(player_id == data['Second Runner']) & ('T' == data['PO For Runner On 2nd Flag'])].shape[0]

    sb += data[(player_id == data['Third Runner']) & ('T' == data['SB For Runner On 3rd Flag'])].shape[0]
    cs += data[(player_id == data['Third Runner']) & ('T' == data['CS For Runner On 3rd Flag'])].shape[0]
    po += data[(player_id == data['Third Runner']) & ('T' == data['PO For Runner On 3rd Flag'])].shape[0]
    # calculate errors
    err = 0
    err += data[(player_id == data['1st Error Player'])].shape[0]
    err += data[(player_id == data['2nd Error Player'])].shape[0]
    err += data[(player_id == data['3rd Error Player'])].shape[0]
    # fill new dictionary with stats
    stats[f'{team} Team Player {player_num} ID'] = [player_id]
    stats[f'{team} Team Player {player_num} Hits'] = [h]
    stats[f'{team} Team Player {player_num} Singles'] = [s]
    stats[f'{team} Team Player {player_num} Doubles'] = [d]
    stats[f'{team} Team Player {player_num} Triples'] = [t]
    stats[f'{team} Team Player {player_num} Home Runs'] = [hr]
    stats[f'{team} Team Player {player_num} Walks'] = [w]
    stats[f'{team} Team Player {player_num} Bunts'] = [b]
    stats[f'{team} Team Player {player_num} Sacrifice Bunts'] = [sh]
    stats[f'{team} Team Player {player_num} Sacrifice Flies'] = [sf]
    stats[f'{team} Team Player {player_num} RBIs'] = [rbi]
    stats[f'{team} Team Player {player_num} At-bats'] = [ab]
    stats[f'{team} Team Player {player_num} Stolen Bases'] = [sb]
    stats[f'{team} Team Player {player_num} Caught Stealing'] = [cs]
    stats[f'{team} Team Player {player_num} Picked Off'] = [po]
    stats[f'{team} Team Player {player_num} Errors'] = [err]
    # return stats
    return stats
    
def pbp_parser(pbp_data, starting_lineups):
    # get column labels new play-by-play dataframes
    defaults = ['Game ID']
    player_stats = ['ID', 'Hits', 'Singles',
                    'Doubles', 'Triples', 'Home Runs', 
                    'Walks', 'Bunts', 'Sacrifice Bunts', 
                    'Sacrifice Flies', 'RBIs', 'At-bats', 
                    'Stolen Bases', 'Caught Stealing', 
                    'Picked Off', 'Errors']
    visiting = list()
    home = list()
    for i in range(9):
        for label in player_stats:
            visiting.append(f'Visiting Team Player {i+1} {label}')
            home.append(f'Home Team Player {i+1} {label}')
    pbp_labels = defaults + visiting + home
    
    # create new data frame
    pbp_data_new = pd.DataFrame(columns=pbp_labels)
    
    current_game_stats = dict()
    for i in range(starting_lineups.shape[0]):
        game_id = starting_lineups.loc[0, 'Game ID']
        current_game_stats.update({'Game ID': game_id})
        current_game_pbp = pbp_data[pbp_data['Game ID'] == game_id]
        for j in range(9):
            vp = starting_lineups.loc[i, f'Visiting Team Player {j+1} ID']
            hp = starting_lineups.loc[i, f'Home Team Player {j+1} ID']
            current_game_stats.update(calculate_stats(data=current_game_pbp, player_id=vp, player_num=j+1, team='Visiting'))
            current_game_stats.update(calculate_stats(data=current_game_pbp, player_id=hp, player_num=j+1, team='Home'))
        pd.concat([pbp_data_new, pd.DataFrame(current_game_stats)], ignore_index=True)
        
    return pbp_data_new

In [184]:
pbp_data_path = 'datasets/retro_sheet_pbp_new/'
pbp_data = []
for file in os.listdir(pbp_data_path):
    if '.EVN' in file or '.EVA' in file:
        pbp_data.append(pd.read_csv(pbp_data_path + file, names=column_labels))
pbp_data = pd.concat(pbp_data, ignore_index=True)

pbp_data

,Game ID,Visiting Team,Batting Team,Batter,First Runner,Second Runner,Third Runner,Event Type,Batter Event Flag,AB Flag,...,SB For Runner On 1st Flag,SB For Runner On 2nd Flag,SB For Runner On 3rd Flag,CS For Runner On 1st Flag,CS For Runner On 2nd Flag,CS For Runner On 3rd Flag,PO For Runner On 1st Flag,PO For Runner On 2nd Flag,PO For Runner On 3rd Flag,Event Num
0,CHN201704100,LAN,0,forsl001,NaN,NaN,NaN,3,T,T,...,F,F,F,F,F,F,F,F,F,1
1,CHN201704100,LAN,0,seagc001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,2
2,CHN201704100,LAN,0,turnj001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,3
3,CHN201704100,LAN,1,schwk001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,4
4,CHN201704100,LAN,1,bryak001,NaN,NaN,NaN,14,T,F,...,F,F,F,F,F,F,F,F,F,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144617,LAN201810010,COL,0,arenn001,NaN,NaN,NaN,23,T,T,...,F,F,F,F,F,F,F,F,F,67
1144618,LAN201810010,COL,0,stort001,NaN,NaN,NaN,23,T,T,...,F,F,F,F,F,F,F,F,F,68
1144619,LAN201810010,COL,0,gonzc001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,69
1144620,LAN201810010,COL,0,desmi001,NaN,NaN,NaN,3,T,T,...,F,F,F,F,F,F,F,F,F,70


In [189]:
sl_data_path = 'datasets/starting_lineups/'
sl_data = []
for file in os.listdir(sl_data_path):
    if 'csv' in file:
        sl_data.append(pd.read_csv(sl_data_path + file))
sl_data = pd.concat(sl_data, ignore_index=True).drop(columns='Unnamed: 0')

sl_data

,Game ID,Visiting Team Player 1 ID,Visiting Team Player 2 ID,Visiting Team Player 3 ID,Visiting Team Player 4 ID,Visiting Team Player 5 ID,Visiting Team Player 6 ID,Visiting Team Player 7 ID,Visiting Team Player 8 ID,Visiting Team Player 9 ID,Home Team Player 1 ID,Home Team Player 2 ID,Home Team Player 3 ID,Home Team Player 4 ID,Home Team Player 5 ID,Home Team Player 6 ID,Home Team Player 7 ID,Home Team Player 8 ID,Home Team Player 9 ID
0,ARI201704020,spand001,beltb001,pench001,poseb001,crawb001,nunee002,parkj002,panij002,bumgm001,polla001,owinc001,goldp001,lambj001,tomay001,drurb001,perad001,mathj001,greiz001
1,SLN201704020,schwk001,bryak001,rizza001,zobrb001,russa002,heywj001,contw001,lestj001,baezj001,fowld001,diaza003,carpm002,peraj001,moliy001,piscs001,gyorj001,gricr001,martc006
2,TBA201704020,gardb001,sancg002,birdg001,hollm001,ellsj001,casts001,headc001,judga001,torrr001,dickc002,kierk001,longe001,millb002,souzs001,morrl001,beckt001,smitm007,norrd001
3,CIN201704030,hernc005,kendh001,herro001,franm004,saunm001,joset001,ruppc001,galvf001,hellj001,hamib001,peraj003,vottj001,duvaa001,suare001,sches001,cozaz001,barnt001,felds001
4,LAN201704030,margm001,myerw001,solay001,renfh001,schir001,hedga001,aybae001,chacj001,jankt001,tolea001,seagc001,turnj001,gonza003,forsl001,pedej001,grany001,puigy001,kersc001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14572,MIN201809300,sancc001,garcl004,palkd001,garca003,davim005,delmn001,castw002,moncy001,cordr001,mauej001,polaj001,cavej001,grosr001,garvm001,keplm001,forsl001,astuw001,gratj001
14573,SEA201809300,choos001,andre001,odorr001,belta001,gallj002,guzmr001,perec003,calhw001,robid003,hereg002,spand001,cruzn002,seagk001,voged001,healr001,negrk001,freid001,romia001
14574,TBA201809300,pillk001,gricr001,mckib001,hernt002,tellr001,diaza003,mcgur002,bertj001,urenr001,smitm007,phamt001,wendj002,cronc002,loweb001,adamw002,meada001,bauej001,ciufn001
14575,CHN201810010,cainl001,yelic001,braur002,aguij001,schoj001,mousm001,krate001,arcio002,chacj001,murpd006,zobrb001,baezj001,rizza001,bryak001,schwk001,heywj001,quinj001,contw001


In [208]:
game_id = mets_starting_lineups.loc[0, 'Game ID']
current_game_stats = dict()
current_game_stats.update({'Game ID': game_id})
current_game_pbp = mets_2014[(game_id == mets_2014['Game ID'])]
for j in range(9):
    vp = mets_starting_lineups.loc[0, f'Visiting Team Player {j+1} ID']
    current_game_stats.update(calculate_stats(data=current_game_pbp, player_id=vp, player_num=j+1, team='Visiting'))
for j in range(9):
    hp = mets_starting_lineups.loc[0, f'Home Team Player {j+1} ID']
    current_game_stats.update(calculate_stats(data=current_game_pbp, player_id=hp, player_num=j+1, team='Home'))

game_1 = pd.DataFrame(current_game_stats)

In [209]:
game_id = mets_starting_lineups.loc[1, 'Game ID']
current_game_stats = dict()
current_game_stats.update({'Game ID': game_id})
current_game_pbp = mets_2014[(game_id == mets_2014['Game ID'])]
for j in range(9):
    vp = mets_starting_lineups.loc[1, f'Visiting Team Player {j+1} ID']
    current_game_stats.update(calculate_stats(data=current_game_pbp, player_id=vp, player_num=j+1, team='Visiting'))
for j in range(9):
    hp = mets_starting_lineups.loc[1, f'Home Team Player {j+1} ID']
    current_game_stats.update(calculate_stats(data=current_game_pbp, player_id=hp, player_num=j+1, team='Home'))

game_2 = pd.DataFrame(current_game_stats)

pd.concat([game_1,game_2])

,Game ID,Visiting Team Player 1 ID,Visiting Team Player 1 Hits,Visiting Team Player 1 Singles,Visiting Team Player 1 Doubles,Visiting Team Player 1 Triples,Visiting Team Player 1 Home Runs,Visiting Team Player 1 Walks,Visiting Team Player 1 Bunts,Visiting Team Player 1 Sacrifice Bunts,...,Home Team Player 9 Walks,Home Team Player 9 Bunts,Home Team Player 9 Sacrifice Bunts,Home Team Player 9 Sacrifice Flies,Home Team Player 9 RBIs,Home Team Player 9 At-bats,Home Team Player 9 Stolen Bases,Home Team Player 9 Caught Stealing,Home Team Player 9 Picked Off,Home Team Player 9 Errors
0,NYN201403310,spand001,1,0,1,0,0,1,0,0,...,0,1,1,0,0,1,0,0,0,0
0,NYN201404020,spand001,2,0,1,1,0,0,0,0,...,0,1,0,0,0,2,0,0,0,0


In [212]:
defaults = ['Game ID', 'Home', 'Visitors']
player_stats = ['ID', 'Hits', 'Singles',
                'Doubles', 'Triples', 'Home Runs', 
                'Walks', 'Bunts', 'Sacrifice Bunts', 
                'Sacrifice Flies', 'RBIs', 'At-bats', 
                'Stolen Bases', 'Caught Stealing', 
                'Picked Off', 'Errors']
visiting = list()
home = list()
for i in range(9):
    for label in player_stats:
        visiting.append(f'Visiting Team Player {i+1} {label}')
        home.append(f'Home Team Player {i+1} {label}')
pbp_labels = defaults + visiting + home

# create new data frame
pbp_data_new = pd.DataFrame(columns=pbp_labels)

pd.concat([game_1, game_2, pbp_data_new])

,Game ID,Visiting Team Player 1 ID,Visiting Team Player 1 Hits,Visiting Team Player 1 Singles,Visiting Team Player 1 Doubles,Visiting Team Player 1 Triples,Visiting Team Player 1 Home Runs,Visiting Team Player 1 Walks,Visiting Team Player 1 Bunts,Visiting Team Player 1 Sacrifice Bunts,...,Home Team Player 9 Sacrifice Bunts,Home Team Player 9 Sacrifice Flies,Home Team Player 9 RBIs,Home Team Player 9 At-bats,Home Team Player 9 Stolen Bases,Home Team Player 9 Caught Stealing,Home Team Player 9 Picked Off,Home Team Player 9 Errors,Home,Visitors
0,NYN201403310,spand001,1,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,0,NaN,NaN
0,NYN201404020,spand001,2,0,1,1,0,0,0,0,...,0,0,0,2,0,0,0,0,NaN,NaN


In [49]:
starting_lineups_2014 = pd.read_csv('datasets/starting_lineups/SL2014.csv')

In [77]:
mets_starting_lineups = starting_lineups_2014[['NYN' in game_id for game_id in starting_lineups_2014['Game ID']]]

mets_starting_lineups

,Unnamed: 0,Game ID,Visiting Team Player 1 ID,Visiting Team Player 2 ID,Visiting Team Player 3 ID,Visiting Team Player 4 ID,Visiting Team Player 5 ID,Visiting Team Player 6 ID,Visiting Team Player 7 ID,Visiting Team Player 8 ID,Visiting Team Player 9 ID,Home Team Player 1 ID,Home Team Player 2 ID,Home Team Player 3 ID,Home Team Player 4 ID,Home Team Player 5 ID,Home Team Player 6 ID,Home Team Player 7 ID,Home Team Player 8 ID,Home Team Player 9 ID
14,14,NYN201403310,spand001,zimmr001,wertj001,ramow001,harpb003,desmi001,laroa001,renda001,stras001,youne003,lagaj001,wrigd002,granc001,browa003,davii001,darnt001,tejar001,gee-d001
37,37,NYN201404020,spand001,renda001,wertj001,laroa001,zimmr001,harpb003,desmi001,lobaj001,gonzg003,lagaj001,florw001,wrigd002,granc001,younc004,satij001,darnt001,tejar001,colob001
48,48,NYN201404030,spand001,harpb003,wertj001,laroa001,zimmr001,desmi001,espid001,leons001,roart001,youne003,murpd006,wrigd002,granc001,dudal001,lagaj001,darnt001,tejar001,wheez001
61,61,NYN201404040,bernr001,philb001,vottj001,brucj001,ludwr001,frazt001,penab002,santr002,leakm001,youne003,murpd006,wrigd002,granc001,dudal001,lagaj001,darnt001,tejar001,mejij001
76,76,NYN201404050,bernr001,philb001,vottj001,brucj001,ludwr001,frazt001,cozaz001,barnt001,cuetj001,youne003,murpd006,wrigd002,granc001,dudal001,lagaj001,recka001,tejar001,gee-d001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,2258,NYN201409160,yelic001,solad001,mcgec001,ozunm001,joneg002,saltj001,johnr008,lucae001,eovan001,lagaj001,murpd006,darnt001,dudal001,florw001,granc001,dendm001,tejar001,colob001
2273,2273,NYN201409170,yelic001,solad001,mcgec001,ozunm001,joneg002,johnr008,hecha001,mathj001,alvah001,dendm001,murpd006,darnt001,dudal001,florw001,granc001,nieuk001,tejar001,gee-d001
2395,2395,NYN201409260,grosr001,altuj001,cartc002,fowld001,domim001,marij002,villj001,corpc001,peacb001,youne003,florw001,murpd006,dudal001,granc001,dendm001,recka001,tejar001,niesj001
2411,2411,NYN201409270,grosr001,altuj001,cartc002,fowld001,castj006,gonzm002,domim001,marij002,dedus001,dendm001,youne003,murpd006,dudal001,granc001,florw001,recka001,tejar001,montr004


In [78]:
mets_starting_lineups.shape

(81, 20)

In [79]:
mets_starting_lineups.reset_index(inplace=True)

In [83]:
mets_starting_lineups

,Game ID,Visiting Team Player 1 ID,Visiting Team Player 2 ID,Visiting Team Player 3 ID,Visiting Team Player 4 ID,Visiting Team Player 5 ID,Visiting Team Player 6 ID,Visiting Team Player 7 ID,Visiting Team Player 8 ID,Visiting Team Player 9 ID,Home Team Player 1 ID,Home Team Player 2 ID,Home Team Player 3 ID,Home Team Player 4 ID,Home Team Player 5 ID,Home Team Player 6 ID,Home Team Player 7 ID,Home Team Player 8 ID,Home Team Player 9 ID
0,NYN201403310,spand001,zimmr001,wertj001,ramow001,harpb003,desmi001,laroa001,renda001,stras001,youne003,lagaj001,wrigd002,granc001,browa003,davii001,darnt001,tejar001,gee-d001
1,NYN201404020,spand001,renda001,wertj001,laroa001,zimmr001,harpb003,desmi001,lobaj001,gonzg003,lagaj001,florw001,wrigd002,granc001,younc004,satij001,darnt001,tejar001,colob001
2,NYN201404030,spand001,harpb003,wertj001,laroa001,zimmr001,desmi001,espid001,leons001,roart001,youne003,murpd006,wrigd002,granc001,dudal001,lagaj001,darnt001,tejar001,wheez001
3,NYN201404040,bernr001,philb001,vottj001,brucj001,ludwr001,frazt001,penab002,santr002,leakm001,youne003,murpd006,wrigd002,granc001,dudal001,lagaj001,darnt001,tejar001,mejij001
4,NYN201404050,bernr001,philb001,vottj001,brucj001,ludwr001,frazt001,cozaz001,barnt001,cuetj001,youne003,murpd006,wrigd002,granc001,dudal001,lagaj001,recka001,tejar001,gee-d001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NYN201409160,yelic001,solad001,mcgec001,ozunm001,joneg002,saltj001,johnr008,lucae001,eovan001,lagaj001,murpd006,darnt001,dudal001,florw001,granc001,dendm001,tejar001,colob001
77,NYN201409170,yelic001,solad001,mcgec001,ozunm001,joneg002,johnr008,hecha001,mathj001,alvah001,dendm001,murpd006,darnt001,dudal001,florw001,granc001,nieuk001,tejar001,gee-d001
78,NYN201409260,grosr001,altuj001,cartc002,fowld001,domim001,marij002,villj001,corpc001,peacb001,youne003,florw001,murpd006,dudal001,granc001,dendm001,recka001,tejar001,niesj001
79,NYN201409270,grosr001,altuj001,cartc002,fowld001,castj006,gonzm002,domim001,marij002,dedus001,dendm001,youne003,murpd006,dudal001,granc001,florw001,recka001,tejar001,montr004


In [148]:
game_id = mets_starting_lineups.loc[0, 'Game ID']
player_id = mets_starting_lineups.loc[0, 'Visiting Team Player 1 ID']

player_stats = mets_2014[(game_id == mets_2014['Game ID']) &
                         (player_id == mets_2014['Batter'])]

player_stats

,Game ID,Visiting Team,Batting Team,Batter,First Runner,Second Runner,Third Runner,Event Type,Batter Event Flag,AB Flag,...,SB For Runner On 1st Flag,SB For Runner On 2nd Flag,SB For Runner On 3rd Flag,CS For Runner On 1st Flag,CS For Runner On 2nd Flag,CS For Runner On 3rd Flag,PO For Runner On 1st Flag,PO For Runner On 2nd Flag,PO For Runner On 3rd Flag,Event Num
0,NYN201403310,WAS,0,spand001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,1
20,NYN201403310,WAS,0,spand001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,21
40,NYN201403310,WAS,0,spand001,NaN,NaN,NaN,2,T,T,...,F,F,F,F,F,F,F,F,F,41
52,NYN201403310,WAS,0,spand001,mclon001,renda001,laroa001,14,T,F,...,F,F,F,F,F,F,F,F,F,53
68,NYN201403310,WAS,0,spand001,espid001,desmi001,NaN,21,T,T,...,F,F,F,F,F,F,F,F,F,69


In [149]:
# how to calculate hits, singles, doubles, triple and home runs at the same time
h, s, d, t, hr = 0, 0, 0, 0, 0
for hit_value in player_stats['Hit Value']:
    if hit_value == 1:
        s, h = s+1, h+1
    elif hit_value == 2:
        d, h = d+1, h+1
    elif hit_value == 3:
        t, h = t+1, h+1
    elif hit_value == 4:
        hr, h = hr+1, h+1
        
print(h, s, d, t, hr)

1 0 1 0 0


In [150]:
# calculate walks
w = 0
for event_type in player_stats['Event Type']:
    if event_type == 14 or event_type == 15:
        w += 1
        
print(w)

1


In [151]:
# calculate at bats, sac hits, sac flies, rbis and bunts
tf = ['AB Flag', 'SH Flag', 'SF Flag', 'RBI On Play', 'Bunt Flag']
tf_val = [0,0,0,0,0]
for label_index in range(len(tf)):
    for event in player_stats[tf[label_index]]:
        if type(event) == str and event == 'T':
            tf_val[label_index] += 1
        elif type(event) == int and event > 0:
            tf_val[label_index] += event

print(tf_val)

[4, 0, 0, 2, 0]


In [155]:
player_stats['T' == player_stats['AB Flag']].shape[0]

4

In [113]:
player_stats.loc[68, 'RBI On Play']

1

In [158]:
# player's base running stats
sb, cs, po = 0, 0, 0
sb += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['First Runner']) & ('T' == mets_2014['SB For Runner On 1st Flag'])].shape[0]
cs += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['First Runner']) & ('T' == mets_2014['CS For Runner On 1st Flag'])].shape[0]
po += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['First Runner']) & ('T' == mets_2014['PO For Runner On 1st Flag'])].shape[0]

sb += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['Second Runner']) & ('T' == mets_2014['SB For Runner On 2nd Flag'])].shape[0]
cs += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['Second Runner']) & ('T' == mets_2014['CS For Runner On 2nd Flag'])].shape[0]
po += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['Second Runner']) & ('T' == mets_2014['PO For Runner On 2nd Flag'])].shape[0]

sb += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['Third Runner']) & ('T' == mets_2014['SB For Runner On 3rd Flag'])].shape[0]
cs += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['Third Runner']) & ('T' == mets_2014['CS For Runner On 3rd Flag'])].shape[0]
po += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['Third Runner']) & ('T' == mets_2014['PO For Runner On 3rd Flag'])].shape[0]

print(sb, cs, po)

0 0 0


In [138]:
# calculate errors
err = 0
err += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['1st Error Player'])].shape[0]
err += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['2nd Error Player'])].shape[0]
err += mets_2014[(game_id == mets_2014['Game ID']) & (player_id == mets_2014['3rd Error Player'])].shape[0]

print(err)

0


In [120]:
defaults = ['Game ID', 'Visiting Team', 'Home Team']
player_stats = ['ID', 'Hits', 'Singles',
                    'Doubles', 'Triples', 'Home Runs', 
                    'Walks', 'Bunts', 'Sacrifice Hits', 
                    'Sacrifice Flies', 'RBIs', 'At-bats', 
                    'Stolen Bases', 'Caught Stealing', 
                    'Picked Off', 'Errors']
visiting = list()
home = list()
for i in range(9):
    for label in player_stats:
        visiting.append(f'Visiting Team Player {i+1} {label}')
        home.append(f'Home Team Player {i+1} {label}')
pbp_labels = defaults + visiting + home
    
# create new data frame
pbp_data_new = pd.DataFrame(columns=pbp_labels)

pbp_data_new

,Game ID,Visiting Team,Home Team,Visiting Team Player 1 ID,Visiting Team Player 1 Hits,Visiting Team Player 1 Singles,Visiting Team Player 1 Doubles,Visiting Team Player 1 Triples,Visiting Team Player 1 Home Runs,Visiting Team Player 1 Walks,...,Home Team Player 9 Walks,Home Team Player 9 Bunts,Home Team Player 9 Sacrifice Hits,Home Team Player 9 Sacrifice Flies,Home Team Player 9 RBIs,Home Team Player 9 At-bats,Home Team Player 9 Stolen Bases,Home Team Player 9 Caught Stealing,Home Team Player 9 Picked Off,Home Team Player 9 Errors


In [129]:
sl_path = 'datasets/starting_lineups/'
ignore = ['.ipynb_checkpoints']

sl = list()

for file_num in range(2014, 2020, 1):
    sl.append(pd.read_csv(f'datasets/starting_lineups/SL{file_num}.csv'))  
        
starting_lineups = pd.concat(sl, ignore_index=True).drop(columns='Unnamed: 0')

starting_lineups

,Game ID,Visiting Team Player 1 ID,Visiting Team Player 2 ID,Visiting Team Player 3 ID,Visiting Team Player 4 ID,Visiting Team Player 5 ID,Visiting Team Player 6 ID,Visiting Team Player 7 ID,Visiting Team Player 8 ID,Visiting Team Player 9 ID,Home Team Player 1 ID,Home Team Player 2 ID,Home Team Player 3 ID,Home Team Player 4 ID,Home Team Player 5 ID,Home Team Player 6 ID,Home Team Player 7 ID,Home Team Player 8 ID,Home Team Player 9 ID
0,ARI201403220,puigy001,turnj001,ramih003,gonza003,vanss001,uribj002,ethia001,ellia001,kersc001,polla001,hilla001,goldp001,pradm001,trumm001,montm001,owinc001,parrg001,milew001
1,ARI201403230,gordd002,puigy001,ramih003,gonza003,ethia001,ellia001,baxtm001,uribj002,ryu-h001,polla001,hilla001,goldp001,pradm001,montm001,trumm001,parrg001,gregd001,cahit001
2,SDN201403300,crawc002,puigy001,ramih003,gonza003,ethia001,uribj002,ellia001,gordd002,ryu-h001,cabre001,denoc001,headc001,gyorj001,alony001,medit001,venaw001,river003,casha001
3,ANA201403310,almoa001,millb002,canor001,smoaj001,morrl001,seagk001,saunm001,ackld001,zunim001,calhk001,troum001,pujoa001,hamij003,freed001,ibanr001,kendh001,iannc001,aybae001
4,BAL201403310,navad002,pedrd001,ortid001,napom001,carpm001,sizeg001,bogax001,piera001,middw001,markn001,hardj003,jonea003,davic003,cruzn002,wietm001,yound003,flahr001,schoj001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14572,CHA201909290,reyev001,mercj002,cabrm001,hickj001,rodrr009,stewc002,demet001,greig001,castw003,sancc001,andet001,abrej003,moncy001,jimee001,collz001,castw002,palkd001,engea001
14573,KCA201909290,wadel001,polaj001,sanom001,cronc002,cavej001,schoj001,castj006,torrr001,milli001,merrw001,solej001,dozih001,gorda001,mcbrr001,cuthc001,mejie001,arteh001,dinin001
14574,SEA201909290,semim001,profj001,piscs001,davik003,brows003,phegj001,neuss001,barrf001,bolts001,longs001,crawj002,nolaa002,seagk001,lewik001,narvo001,voged001,smitm007,gordd002
14575,TEX201909290,lemad001,judga001,gardb001,stanm004,torrg001,sancg002,gregd001,urshg001,maybc001,choos001,andre001,calhw001,santd001,odorr001,solan001,guzmr001,deshd002,trevj001
